# Форматы данных (1)

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [ ]:
import json

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

for user in data[:3]:
    print(user)


1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [ ]:
import re

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

domains = set()

for user in data:
    match = re.search(r'@([\w.]+)', user['mail'])
    if match:
        domains.add(match.group(1))

for domain in domains:
    print(domain)

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [ ]:
def find_user(username):
    with open('data/contributors_sample.json', 'r') as f:
        data = json.load(f)

    for user in data:
        if user['username'] == username:
            return user

    raise ValueError('User with username "{}" not found'.format(username))

user = find_user('vickitaylor')
print(user)

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [ ]:
with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

male_count = 0
female_count = 0

for user in data:
    if user['sex'] == 'M':
        male_count += 1
    elif user['sex'] == 'F':
        female_count += 1

print('Number of males:', male_count)
print('Number of females:', female_count)

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [ ]:
import pandas as pd

with open('data/contributors_sample.json', 'r') as f:
    data = json.load(f)

contributors = pd.DataFrame(data, columns=['id', 'username', 'sex'])

print(contributors.head())

1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [ ]:
contributors = pd.read_json('data/contributors_sample.json')
recipes = pd.read_csv('data/recipes_sample.csv')

merged = pd.merge(recipes, contributors, left_on='contributor_id', right_on='id', how='left')

missing_count = merged[merged['username'].isnull()]['contributor_id'].nunique()
print('Number of missing contributors:', missing_count)

Number of missing contributors: 4204


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [ ]:
import pickle

positions = {}

for user in data:
    for position in user['jobs']:
        if position not in positions:
            positions[position] = [user['username']]
        else:
            positions[position].append(user['username'])

with open('positions.pickle', 'wb') as f:
    pickle.dump(positions, f)

for job, usernames in positions.items():
    print(job, usernames[:3])

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
jobs_dict = {}

for user in data:
    for job in user['jobs']:
        if job not in jobs_dict:
            jobs_dict[job] = []
        jobs_dict[job].append(user['username'])

# Сохраняем в формате pickle
with open('job_people.pickle', 'wb') as f:
    pickle.dump(jobs_dict, f)

# Сохраняем в формате JSON
with open('job_people.json', 'w') as f:
    json.dump(jobs_dict, f, indent=4)

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open('job_people.pickle', 'rb') as f:
    jobs_dict = pickle.load(f)

for job, usernames in jobs_dict.items():
    print(job, usernames[:3])

Если файл job_people.pickle был записан корректно в предыдущем задании, этот код должен вывести содержимое словаря jobs_dict, который был создан на основе файла contributors_sample.json и содержит информацию о том, какие люди занимали какие должности.

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
import xml.etree.ElementTree as ET

tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    recipe_id = recipe.get('id')
    steps = [step.text.strip() for step in recipe.findall('step')]
    steps_dict[recipe_id] = steps

with open('steps_sample.json', 'w') as f:
    json.dump(steps_dict, f)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

steps_dict = {}

for recipe in root.findall('recipe'):
    id_prescription = recipe.find('id').text
    steps = recipe.find('steps').findall('step')
    num_steps = len(steps)
    if num_steps not in steps_dict:
        steps_dict[num_steps] = [id_prescription]
    else:
        steps_dict[num_steps].append(id_prescription)

for step, id in steps_dict.items():
    print(step, id[:3])

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

recipes_with_time = []

for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.findall('steps/step')
    for step in steps:
        if 'has_minutes' in step.attrib or 'has_hours' in step.attrib:
            recipes_with_time.append(recipe_id)
            break

print(recipes_with_time[:30])

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
import csv

# Load data from CSV file
recipes = pd.read_csv('data/recipes_sample.csv')

# Load data from XML file
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

# Create dictionary of steps for each recipe
steps_dict = {}
for recipe in root.findall('recipe'):
    recipe_id = recipe.find('id').text
    steps = recipe.findall('steps/step')
    steps_dict[recipe_id] = len(steps)

# Fill missing n_steps values
for i, row in recipes.iterrows():
    recipe_id = row['id']
    n_steps = row['n_steps']
    if pd.isna(n_steps):
        if recipe_id in steps_dict:
            recipes.at[i, 'n_steps'] = steps_dict[recipe_id]

# Display the updated DataFrame
print(recipes)

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
# Load recipes data
recipes = pd.read_csv('data/recipes_sample.csv')

# Check for missing values
if recipes['n_steps'].isnull().any():
    print('Column "n_steps" contains missing values')
else:
    # Convert n_steps to integer type
    recipes['n_steps'] = recipes['n_steps'].astype(int)
    # Save to file
    recipes.to_csv('recipes_sample_with_filled_nsteps.csv', index=False)

print(recipes)